In [1]:
import sys
sys.path.append('../')

from src.utils.video import VideoIterator

import face_recognition
import cv2
import numpy as np
from tqdm.auto import tqdm
from moviepy.editor import VideoFileClip

from warnings import filterwarnings
filterwarnings('ignore')

FRAME_COMPRESSION = 1
FRAME_SKIP = 4

In [2]:
input_video_path = '../data/Learn English With Street Interviews! #40 What Makes you Happy_#britishenglish #howtospeakenglish.mp4'
input_image_paths = [
    '../data/Снимок экрана 2023-05-25 в 14.52.39.png',
]
output_video_path = '../data/output3.mp4'

In [3]:
input_face_encodings = []
for input_image_path in tqdm(input_image_paths):
    input_image = face_recognition.load_image_file(input_image_path)
    input_face_encodings.extend(face_recognition.face_encodings(input_image, num_jitters=10))

  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
video_iterator = VideoIterator(input_video_path)
video_writer = cv2.VideoWriter(
    output_video_path,
    cv2.VideoWriter_fourcc(*'MP4V'),
    video_iterator.fps,
    (video_iterator.width, video_iterator.height),
)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [5]:
face_locations = []
face_encodings = []
known_faces = []

for i, frame in enumerate(tqdm(video_iterator)):
    if i % FRAME_SKIP == 0:
        small_frame = cv2.cvtColor(cv2.resize(frame, (0, 0), fx=1/FRAME_COMPRESSION, fy=1/FRAME_COMPRESSION), cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)

        known_faces = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(input_face_encodings, face_encoding, tolerance=0.5)
            known_faces.append(True if True in matches else False)
    
    # Create mask for blurred faces
    mask = np.zeros_like(frame) - 1
    for (top, right, bottom, left), known in zip(face_locations, known_faces):
        if known:
            continue
        top *= FRAME_COMPRESSION
        right *= FRAME_COMPRESSION
        bottom *= FRAME_COMPRESSION
        left *= FRAME_COMPRESSION
        mask[top:bottom, left:right] = 0
    # Blur frame on mask
    blurred_frame = cv2.GaussianBlur(frame, (45, 45), 0)
    frame = np.where(
        mask==np.array([255, 255, 255]),
        frame,
        blurred_frame
    )

    # Write frame to output video
    video_writer.write(frame)

video_writer.release()

  0%|          | 0/6860 [00:00<?, ?it/s]

# Sound

In [6]:
input_video_path = '../data/Learn English With Street Interviews! #40 What Makes you Happy_#britishenglish #howtospeakenglish.mp4'
output_video_path = '../data/output3.mp4'

In [7]:
input_clip = VideoFileClip(input_video_path)
output_clip = VideoFileClip(output_video_path)
output_clip = output_clip.set_audio(input_clip.audio)

In [8]:
output_clip.write_videofile(
    '../data/final3.mp4',
    codec='libx264', 
    audio_codec='aac', 
    temp_audiofile='temp-audio.m4a', 
    remove_temp=True,
)

Moviepy - Building video ../data/final3.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video ../data/final3.mp4



Moviepy - Done !
Moviepy - video ready ../data/final3.mp4
